In [1]:

import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
import pandas as pd

# CONFIGURACIÓN
WRITE_API_KEY = '3CNCDCYFJXGA94I3'
READ_API_KEY = 'QTOJJH6QRLXJ0627'
CHANNEL_ID = '2960836'

# ----------- PESTAÑA 1: ENVIAR DATOS -----------
co2 = widgets.FloatText(description='CO₂ (ppm):')
ruido = widgets.FloatText(description='Ruido (dB):')
luz = widgets.FloatText(description='Luz (lux):')
presion = widgets.FloatText(description='Presión (hPa):')
viento = widgets.FloatText(description='Viento (km/h):')
btn_enviar = widgets.Button(description="Enviar", button_style='primary')
salida_envio = widgets.Output()

def enviar_datos(b):
    with salida_envio:
        clear_output()
        print("▶ Ejecutando envío...")
        datos = {
            'api_key': WRITE_API_KEY,
            'field1': co2.value,
            'field2': ruido.value,
            'field3': luz.value,
            'field4': presion.value,
            'field5': viento.value
        }
        r = requests.get("https://api.thingspeak.com/update", params=datos)
        print("Código:", r.status_code)
        print("Respuesta:", r.text)
        if r.status_code == 200 and r.text != '0':
            print("✅ Datos enviados correctamente.")
        else:
            print("❌ Error al enviar.")

btn_enviar.on_click(enviar_datos)

tab_enviar = widgets.VBox([
    widgets.HTML(value="<h3>Enviar Datos a ThingSpeak</h3>"),
    co2, ruido, luz, presion, viento,
    btn_enviar,
    salida_envio
])

# ----------- PESTAÑA 2: LEER ÚLTIMO DATO -----------
btn_leer = widgets.Button(description="Leer último dato", button_style='info')
salida_leer = widgets.Output()

def leer_datos(b):
    with salida_leer:
        clear_output()
        print("▶ Consultando último dato registrado...")
        url = f"https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.json?api_key={READ_API_KEY}&results=1"
        r = requests.get(url)
        datos = r.json()
        if datos['feeds']:
            f = datos['feeds'][0]
            print("📊 Último dato:")
            print(f"CO₂: {f['field1']} ppm")
            print(f"Ruido: {f['field2']} dB")
            print(f"Luz: {f['field3']} lux")
            print(f"Presión: {f['field4']} hPa")
            print(f"Viento: {f['field5']} km/h")
        else:
            print("⚠️ No hay datos en el canal.")

btn_leer.on_click(leer_datos)

tab_leer = widgets.VBox([
    widgets.HTML(value="<h3>Leer Último Dato Registrado</h3>"),
    btn_leer,
    salida_leer
])

# ----------- PESTAÑA 3: CONSULTAR HISTORIAL (últimos 5, desde 1) -----------
btn_historial = widgets.Button(description="Cargar últimos 5 datos", button_style='success')
salida_historial = widgets.Output()

def consultar_historial(b):
    with salida_historial:
        clear_output()
        print("▶ Consultando historial de los últimos 5 registros...")
        url = f"https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.json?api_key={READ_API_KEY}&results=5"
        r = requests.get(url)
        datos = r.json()
        if datos['feeds']:
            df = pd.DataFrame(datos['feeds'])
            df = df[['created_at', 'field1', 'field2', 'field3', 'field4', 'field5']]
            df.columns = ['Fecha', 'CO₂ (ppm)', 'Ruido (dB)', 'Luz (lux)', 'Presión (hPa)', 'Viento (km/h)']
            df.index += 1  # Cambiar índice para empezar en 1
            display(df)
        else:
            print("⚠️ No hay historial disponible.")

btn_historial.on_click(consultar_historial)

tab_historial = widgets.VBox([
    widgets.HTML(value="<h3>Historial de Datos (últimos 5)</h3>"),
    btn_historial,
    salida_historial
])

# ----------- MOSTRAR PESTAÑAS -----------
tabs = widgets.Tab(children=[tab_enviar, tab_leer, tab_historial])
tabs.set_title(0, 'Enviar Datos')
tabs.set_title(1, 'Leer Último Dato')
tabs.set_title(2, 'Consultar Historial')
display(tabs)
